In [1]:
!pip install llama-index

In [26]:
%set_env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [19]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files=['budgetpdf.pdf']).load_data()

In [4]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes =  splitter.get_nodes_from_documents(documents)

In [5]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [6]:
from llama_index.core import SummaryIndex,VectorStoreIndex
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

In [20]:
summary_query_engine = summary_index.as_query_engine(
    response_mode='tree_summarize',
    use_async=True
)

vector_query_engine = vector_index.as_query_engine()

In [21]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine = summary_query_engine,
    description=(
        "Useful for summarization questions related to union budget"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine = vector_query_engine,
    description=(
        "Useful for retrieving specific context from the union budget."
    ),
)



In [22]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


In [23]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool
    ]
)

In [24]:
response = query_engine.query("What is the summary of the document?")
print(response)

The document provides comprehensive details on the Interim Union Budget 2024-25 presented by the Union Minister for Finance and Corporate Affairs, Smt. Nirmala Sitharaman. It highlights key aspects such as social justice initiatives, welfare schemes for farmers and women entrepreneurs, infrastructure development plans, tax reforms, and projections for economic growth. The budget emphasizes inclusive development, capital expenditure outlay increase, fiscal deficit estimates, FDI inflow, and tax-related announcements aimed at supporting economic growth and employment generation. Additionally, it covers initiatives related to empowering marginalized groups, enhancing rural socio-economic landscape, boosting farmers' income, promoting research and innovation, development of iconic tourist centers, and support for youth in sports and technology. The document underscores the government's commitment to governance, development, and performance, with a focus on inclusive growth, welfare program

In [25]:
response = query_engine.query("What are Tax rationalization efforts over the years?")
print(response)

Tax rationalization efforts over the years include a decrease in corporate income tax rates for existing domestic companies from 30% to 22% and for new manufacturing companies to 15%. Additionally, there has been an increase in the threshold for presumptive taxation for retail businesses and professionals. The government has focused on simplifying tax processes, reducing processing times for tax returns significantly, and introducing measures like faceless assessment and appeal for greater efficiency. Furthermore, reforms in customs have led to reduced import release times at various ports and depots.
